In [7]:
import json
from pprint import pprint
import re
from IPython.core.debugger import set_trace
from tqdm import tqdm

In [8]:
saoke_path = "../ori_data/SAOKE_DATA.json"

In [9]:
with open(saoke_path, "r", encoding = "utf-8") as file_in:
    saoke_data = [json.loads(line) for line in file_in]

In [10]:
len(saoke_data)

46930

In [11]:
saoke_data[0]

{'natural': '主要栖息于低山和山麓地带的次生阔叶林、阔叶林和针阔叶混交林中，也出入于人工林和针叶林，夏季在北方有时可上到海拔1700米的中、高山地带，在南方夏季甚至上到海拔3000米左右的森林中，冬季多下到山麓和邻近平原地带的次生阔叶林、人工林和林缘疏林灌丛，有时也进到果园、道旁和地边树丛、房前屋后和庭院中的树上。',
 'logic': [{'predicate': '主要栖息于',
   'qualifier': '_',
   'object': ['[低山|山麓地带]的[次生阔叶林|阔叶林|针阔叶混交林]中'],
   'place': '_',
   'time': '_',
   'subject': '_'},
  {'predicate': '也出入于',
   'qualifier': '_',
   'object': ['[人工林|针叶林]'],
   'place': '_',
   'time': '_',
   'subject': '_'},
  {'predicate': '有时可上到',
   'qualifier': '_',
   'object': ['海拔1700米的[中|高]山地带'],
   'place': '在北方',
   'time': '夏季',
   'subject': '_'},
  {'predicate': '甚至上到',
   'qualifier': '_',
   'object': ['海拔3000米左右的森林中'],
   'place': '在南方',
   'time': '夏季',
   'subject': '_'},
  {'predicate': '多下到',
   'qualifier': '_',
   'object': ['[山麓|邻近平原地带]的[次生阔叶林|人工林|林缘疏林灌丛]'],
   'place': '_',
   'time': '冬季',
   'subject': '_'},
  {'predicate': '有时也进到',
   'qualifier': '_',
   'object': ['[果园|道旁]|[地边树丛|[房前屋后|庭院中]的树上]'],
   'place': '_',
   'time': '_',
   'subj

In [14]:
special_predicates = set()
count_spo = 0
count_joint_po = 0
count_no_p = 0
count_uncon_p = 0
count_normal_p = 0
count_spe_p = 0

count_normal_spo = 0
count_sp = 0
count_po = 0

for idx, example in tqdm(enumerate(saoke_data)):
#     if idx == 3713:
#         set_trace()
    text = example["natural"]
    selected_spo = []
    for spo in example["logic"]:
        count_spo += 1
        
        ## 谓词
        predicate = spo["predicate"]
        
        # 统计predicate缺失情况
        if predicate == "_":
            count_no_p += 1
#             if spo["object"] == "_":
#                 print(spo)
            
        
        # 统计特殊预定义关系：ISA、DESC情况
        elif re.match("[A-Z]+$", predicate):
            count_spe_p += 1
            special_predicates.add(predicate)
#             if predicate in {'NOT', 'PAM', 'PK', 'VS'}:
#                 print(spo)
#                 print(text)
#                 print("---------------------")
        
        # 统计非连续P的情况
        elif "X" in predicate:
#             if "U" in predicate:
#                 print(predicate)
            count_uncon_p += 1
        
        
        elif "X" not in predicate and not re.match("[A-Z]+$", predicate):
            count_normal_p += 1
            obj = spo["object"][0]
            obj = re.sub(re.escape("."), "\.", obj)
            obj = re.sub(re.escape("|"), ".{1,5}", obj)
            obj = re.sub("[\[\]]", "", obj)
            
#             set_trace()
            # 统计连续普通P紧跟O的情况
            po = "{}{}".format(predicate, obj)
            try: 
                if re.search(po, text):
    #                 print(po)
    #                 print(example)
    #                 print("--------------------")
                    count_joint_po += 1
            except Exception as e:
                pass
            
            if spo["subject"] != "_" and spo["object"] != "_": # 主谓宾
                count_normal_spo += 1
#                 print(spo)
            elif spo["subject"] == "_": # 谓宾 （缺失）
                count_po += 1
#                 print(spo)
            elif spo["object"] == "_": # 主谓（不及物Vi，或缺失）
                count_sp += 1
        else:
            print(predicate)
        
        # 主宾/符合结构
        
        def ent_exist(entity):
            entity = re.sub(re.escape("."), "\.", entity)
            entity = re.sub(re.escape("|"), ".{1,10}", entity)
            entity = re.sub("[\[\]]", "", entity)
            
            exist = re.search(entity, text)
            if not exist:
                set_trace()
            return exist
        
        so_list = []
        if "|" in spo["subject"]:
            if not ent_exist(spo["subject"]):
                
                print(spo)
                print(example)
                print("--------------")
        elif type(spo["object"]) is list and len(spo["object"]) > 0:
            for obj in spo["object"]:
                if "|" in obj:
                    if not ent_exist(obj):
                        print(spo)
                        print(example)
                        print("----------------")
print("joint_po: {}".format(count_joint_po / count_spo))
print("joint_po / normal_p: {}".format(count_joint_po / count_normal_p))
print("no_p: {}".format(count_no_p / count_spo))
print("uncon_p: {}".format(count_uncon_p / count_spo))
print("normal_p: {}".format(count_normal_p / count_spo))
print("special_p: {}".format(count_spe_p / count_spo))
print("joint_po + uncon_p: {}".format((count_joint_po + count_uncon_p) / count_spo))

print("-------------------")
print("count_sp / count_normal_p: {}/{}".format(count_sp, count_normal_p))
print("count_po / count_normal_p: {}/{}".format(count_po, count_normal_p))
print("count_normal_spo / count_normal_p: {}/{}".format(count_normal_spo, count_normal_p))
#             selected_spo.append(spo)
#     print(example["natural"])
#     pprint(selected_spo)

0it [00:00, ?it/s]

> <ipython-input-14-92090a09a4d6>(87)ent_exist()
     85             if not exist:
     86                 set_trace()
---> 87             return exist
     88 
     89         so_list = []

ipdb> entity
'ISO9001.{1,10}ISO14001.{1,10}OHSAS18001管理体系认证'
ipdb> text
'通过了ISO9001（质量）、ISO14001（环境）、OHSAS18001（职业健康与安全）管理体系认证；'
ipdb> q


49it [1:11:04, 87.03s/it]


BdbQuit: 

In [ ]:
# {'BIRTH', 'DEATH', 'DESC', 'IN', 'ISA', 'NOT', 'PAM', 'PK', 'VS'}
special_predicates
# NOT 只有两例 去掉
# PAM、PK、VS都是原文中的谓词

In [ ]:
%debug